In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r drive/My\ Drive/Bin/TechJam2019/* .

In [0]:
!cp -r logs/* drive/My\ Drive/Bin/TechJam2019/logs

In [3]:
!nvidia-smi 

Tue Nov  5 15:42:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, GRU, CuDNNLSTM, CuDNNGRU, Bidirectional, Masking
from keras.layers import Dense, InputLayer, Dropout, BatchNormalization
from keras.layers import Reshape, Flatten, Conv1D, AveragePooling1D, MaxPool1D, Activation, Concatenate
from keras.layers import Conv1D
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split

from tqdm import tqdm, trange, tnrange
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import pad_sequences

In [6]:
from data_gen import create_weekly_kplus, create_credit_seq, create_daily_kplus, create_demographic_data
from util import modified_SMAPE
from losses import SMAPE_loss
from callbacks import EvaluateSMAPE

load data gen


In [0]:
demograpgics = pd.read_csv('datasets/exam-1/demographics.csv')
cc = pd.read_csv('datasets/exam-1/cc.csv')
cc.sort_values(by=['cc_no', 'pos_dt'], inplace=True)
kplus = pd.read_csv('datasets/exam-1/kplus.csv')
kplus.sort_values(by=['id', 'sunday'], inplace=True)
train_set = pd.read_csv('datasets/exam-1/train.csv')
test_set = pd.read_csv('datasets/exam-1/test.csv')
raw_demographics = pd.read_csv('datasets/exam-1/raw_demograhgics.csv')

In [0]:
scaled_kplus = kplus.copy()
kplus_scaler =  StandardScaler()
scaled_kplus[['kp_txn_count', 'kp_txn_amt']] = kplus_scaler.fit_transform(kplus[['kp_txn_count', 'kp_txn_amt']])

In [0]:
cc_persons = pd.read_csv('datasets/exam-1/cc_persons.csv')
cc_scaler = StandardScaler()
cc_persons[['cc_txn_amt', 'count']] = cc_scaler.fit_transform(cc_persons[['cc_txn_amt', 'count']])

In [10]:
train_scaler = StandardScaler()
scaled_train_set = train_set.copy()
scaled_train_set.set_index('id', inplace=True)
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'], axis=2))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


# KPLUS

In [0]:
train_kplus = train_kplus[train_kplus['id'] <= 50000]

In [0]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
822506,1,2018-06-03,-0.567504,-0.211370
822505,1,2018-06-17,-0.567504,-0.207460
100625,2,2018-01-07,-0.399415,-0.218408
100629,2,2018-01-14,-0.231325,-0.198075
100641,2,2018-01-21,-0.399415,-0.218408


In [0]:
padding_value = float(-100)
xs = create_weekly_kplus(train_kplus, padding_value)
ys = scaled_train_set.loc[train_kplus['id'].drop_duplicates().to_numpy()].to_numpy()

In [0]:
base_activation = 'tanh'
model = Sequential()
model.add(InputLayer((xs.shape[1], 2)))
model.add(Masking(padding_value, ))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation))
model.add(Dense(1, activation='linear'))

In [0]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [0]:
model.compile(loss=SMAPE_loss(kplus_scaler), optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25, 2)             0         
_________________________________________________________________
masking_2 (Masking)          (None, 25, 2)             0         
_________________________________________________________________
gru_6 (GRU)                  (None, 25, 50)            7950      
_________________________________________________________________
gru_7 (GRU)                  (None, 25, 50)            15150     
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 50)            0         
_________________________________________________________________
gru_8 (GRU)                  (None, 25, 25)            5700      
_________________________________________________________________
gru_9 (GRU)                  (None, 25, 25)            3825      
__________

In [0]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [0]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=kplus_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir='logs/gru')],
          initial_epoch=0,
         epochs=10000)

Instructions for updating:
Use tf.cast instead.
Train on 27725 samples, validate on 6932 samples
Epoch 1/10000
27725/27725 [==============================] - 58s 2ms/step - loss: 0.0896 - val_loss: 0.0827
Epoch 1 | val-SMAPE: 91.72752684189692
Epoch 2/10000
27725/27725 [==============================] - 71s 3ms/step - loss: 0.0849 - val_loss: 0.0796
Epoch 2 | val-SMAPE: 92.04023577019237
Epoch 3/10000
 4256/27725 [===>..........................] - ETA: 52s - loss: 0.0854

KeyboardInterrupt: 

# Credit

In [0]:
cc_persons.head()

,id,pos_dt,cc_no,cc_txn_amt,count,pos_dt_index
0,1,2018-01-20,98397,0.009926,-0.366991,19
1,1,2018-02-17,196794,0.421319,1.115008,47
2,1,2018-05-13,98397,0.152435,-0.366991,132
3,1,2018-06-14,98397,0.152435,-0.366991,164
4,2,2018-01-04,9740,-0.073428,-0.366991,3


In [0]:
train_cc_persons = cc_persons[cc_persons['id'] <= 50000]

In [0]:
def create_credit_x(cc_persons, padding_value):
    xs = [] # [[cc_txn_amt, count], ...]

    empty_seq = np.ones((max_len_seq, 2), dtype=np.float32) * padding_value
    for _id, group in cc_persons.groupby('id'):
        seq = empty_seq.copy()
        seq[group['pos_dt_index'].to_numpy()] = group[['cc_txn_amt', 'count']].to_numpy()

        xs.append(seq)
    xs = np.asarray(xs)
    return xs

In [0]:
padding_value = float(-100)
xs = create_credit_seq(train_cc_persons, padding_value)
ys = scaled_train_set.loc[train_cc_persons['id'].drop_duplicates().to_numpy()].to_numpy()

In [0]:
xs.shape

(39950, 181, 2)

In [0]:
ys.shape

(39950, 1)

In [0]:
base_activation = 'tanh'
RNN = GRU
model = Sequential()
model.add(InputLayer((xs.shape[1] ,2)))
model.add(Masking(padding_value))
model.add(Bidirectional(RNN(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(RNN(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(RNN(32, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(RNN(32, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(RNN(16, )))
model.add(Activation(base_activation))
model.add(Dense(1, activation='linear'))

In [0]:
model = load_model('logs/cc_gru_masking/weights.01-90.25.hdf5')

KeyError: 0

In [0]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [0]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 181, 2)            0         
_________________________________________________________________
masking_4 (Masking)          (None, 181, 2)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 181, 128)          25728     
_________________________________________________________________
activation_6 (Activation)    (None, 181, 128)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 181, 128)          74112     
_________________________________________________________________
activation_7 (Activation)    (None, 181, 128)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 181, 128)          0         
__________

In [0]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [0]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir='logs/cc_gru')],
          initial_epoch=0,
         epochs=10000)

NameError: name 'model' is not defined

In [0]:
model.save('logs/cc_gru_smape/smape:89.635.h5')

# KPLUS + Credit

In [11]:
kplus_n_threshold = 0
kplus_n_ids = scaled_kplus.groupby('id').size()
kplus_n_ids = set(kplus_n_ids[kplus_n_ids >= kplus_n_threshold].keys())
len(kplus_n_ids)

49269

In [12]:
cc_n_threshold = 0
cc_n_ids = cc_persons.groupby('id').size()
cc_n_ids = set(cc_n_ids[cc_n_ids >= cc_n_threshold].keys())
len(cc_n_ids)

51467

In [13]:
kplus_ids = set(scaled_kplus[scaled_kplus['id'] <= 50000]['id'].drop_duplicates())
cc_ids = set(cc_persons[cc_persons['id'] <= 50000]['id'].drop_duplicates())
print(len(kplus_ids), len(cc_ids), len(kplus_ids.union(cc_ids)), len(kplus_ids.intersection(cc_ids)))
ids = sorted(list(kplus_ids.intersection(cc_ids, kplus_n_ids, cc_n_ids)))
ids_df = pd.DataFrame(ids, columns=['id'])

38056 39950 47174 30832


In [0]:
train_kplus = pd.merge(scaled_kplus, ids_df, on='id', how='right')
train_cc_persons = pd.merge(cc_persons, ids_df, on='id', how='right')

In [0]:
padding_value = float(-100)
kplus_padding = cc_padding = padding_value
kplus_padding = kplus_scaler.transform([[0,0]])
cc_padding = cc_scaler.transform([[0,0]])

kplus_xs = create_daily_kplus(train_kplus, kplus_padding)
cc_persons_xs = create_credit_seq(train_cc_persons, cc_padding)


In [0]:

xs = np.concatenate((kplus_xs, cc_persons_xs), axis=2)
# xs = np.concatenate((np.expand_dims(kplus_xs[:, :, 0], axis=2), np.expand_dims(cc_persons_xs[:, :, 0], axis=2)), axis=2)
ys = scaled_train_set.loc[ids]['income'].to_numpy()

In [0]:
base_activation = 'tanh'
RNN = CuDNNLSTM
model = Sequential()
# model.add(Masking(padding_value, ))
model.add((RNN(64, return_sequences=True, input_shape=xs.shape[1:])))
model.add(Activation(base_activation))
model.add((RNN(64, return_sequences=True)))
model.add(Activation(base_activation))

# model.add(Dropout(0.5))
# model.add(RNN(64, return_sequences=True))
# model.add(Activation(base_activation))
model.add(RNN(32, ))
model.add(Activation(base_activation))
# model.add(Dense(10))
# model.add(Dropout(0.25))


model.add(Dense(1, activation='linear'))

In [38]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, strides=1, input_shape=xs.shape[1:], activation='relu'))
model.add(Conv1D(64, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPool1D())
model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPool1D())
model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPool1D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(40))
# model.add(Dropout(0.25))
model.add(Dense(1))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [39]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=Adam() )
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 179, 64)           832       
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 177, 64)           12352     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 88, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 88, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 86, 16)            3088      
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 84, 16)            784       
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 42, 16)           

In [0]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [41]:
log_dir = 'logs/kplus_cc_conv_maxpool3'

model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir),
                     ModelCheckpoint(log_dir + '/ep{epoch}-val_SMAPE:{val_SMAPE}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Train on 24665 samples, validate on 6167 samples
Epoch 1/100
24665/24665 [==============================] - 9s 346us/step - loss: 0.1073 - val_loss: 0.0977
Epoch 1 | val-SMAPE: 90.23215018100439
Epoch 2/100
24665/24665 [==============================] - 8s 328us/step - loss: 0.1015 - val_loss: 0.0909
Epoch 2 | val-SMAPE: 90.91235324776653
Epoch 3/100
24665/24665 [==============================] - 8s 329us/step - loss: 0.0949 - val_loss: 0.0876
Epoch 3 | val-SMAPE: 91.2405387963722
Epoch 4/100
24665/24665 [==============================] - 8s 331us/step - loss: 0.0936 - val_loss: 0.0893
Epoch 4 | val-SMAPE: 91.0732624603812
Epoch 5/100
24665/24665 [==============================] - 8s 333us/step - loss: 0.0928 - val_loss: 0.0866
Epoch 5 | val-SMAPE: 91.3422172638727
Epoch 6/100
24665/24665 [==============================] - 8s 333us/step - loss: 0.0923 - val_loss: 0.0857
Epoch 6 | val-SMAPE: 91.43027230887006
Epoch 7/100
24665/24665 [==============================] - 8s 332us/step - los

KeyboardInterrupt: ignored

In [0]:
model.save('logs/kplus_cc_lstm/ep6-val_SMAPE:90.3078.h5')

# DEMOGRAPHIC + CC + KPLUS

In [61]:
train_set['id'].to_numpy()

array([    1,     2,     3, ..., 49998, 49999, 50000])

In [0]:

def create_all_x_data(ids):
  ids_df = pd.DataFrame(ids, columns=['id'])
  train_kplus = pd.merge(scaled_kplus, ids_df, on='id', how='right')
  train_cc_persons = pd.merge(cc_persons, ids_df, on='id', how='right')

  padding_value = float(-100)
  kplus_padding = cc_padding = padding_value
  kplus_padding = kplus_scaler.transform([[0,0]])
  cc_padding = cc_scaler.transform([[0,0]])
  kplus_xs = create_daily_kplus(train_kplus, kplus_padding)
  cc_persons_xs = create_credit_seq(train_cc_persons, cc_padding)
  transaction_xs = np.concatenate((kplus_xs, cc_persons_xs), axis=2)

  demographic_xs = create_demographic_data(raw_demographics.set_index('id').loc[ids])

  
  return demographic_xs, transaction_xs

In [0]:
def create_all_y_data(ids):
  return scaled_train_set.loc[ids]['income'].to_numpy()

In [46]:
demographic_xs.shape, transaction_xs.shape, ys.shape

((50000, 21), (50000, 181, 4), (50000,))

In [0]:
demo_x_train, demo_x_val, transaction_x_train, transaction_x_val, y_train, y_val \
= train_test_split(demographic_xs, transaction_xs, ys, test_size=0.2)

In [48]:
demo_x_train.shape

(40000, 21)

In [0]:
transaction_model = load_model('/content/logs/kplus_cc_conv_maxpool3/ep14-val_SMAPE:91.58859339541026.h5'
                               , custom_objects={'loss_func':SMAPE_loss(train_scaler)})

In [50]:
transaction_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 179, 64)           832       
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 177, 64)           12352     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 88, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 88, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 86, 16)            3088      
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 84, 16)            784       
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 42, 16)           

In [0]:
for layer in transaction_model.layers:
    layer.trainable = True

In [0]:
demographic_model = Sequential()
demographic_model.add(Dense(20, activation='relu', input_shape=(21,)))
demographic_model.add(Dense(20, activation='relu'))
demographic_model.add(Dense(10, activation='relu'))
demographic_model.add(Dense(1))

In [52]:
demographic_model.compile(optimizer='adam', loss=SMAPE_loss(train_scaler))
demographic_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 20)                440       
_________________________________________________________________
dense_16 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 1,081
Trainable params: 1,081
Non-trainable params: 0
_________________________________________________________________


In [53]:
demographic_model.fit(demo_x_train, y_train,
          validation_data=(demo_x_val, y_val),
          epochs=100)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 7s 171us/step - loss: 0.1056 - val_loss: 0.1045
Epoch 2/100
40000/40000 [==============================] - 6s 143us/step - loss: 0.1027 - val_loss: 0.1043
Epoch 3/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1027 - val_loss: 0.1036
Epoch 4/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1025 - val_loss: 0.1038
Epoch 5/100
40000/40000 [==============================] - 6s 145us/step - loss: 0.1024 - val_loss: 0.1037
Epoch 6/100
40000/40000 [==============================] - 6s 143us/step - loss: 0.1024 - val_loss: 0.1039
Epoch 7/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1024 - val_loss: 0.1039
Epoch 8/100
21440/40000 [===============>..............] - ETA: 2s - loss: 0.1016

KeyboardInterrupt: ignored

In [0]:
for layer in demographic_model.layers:
  layer.trainable = True

In [0]:
x = Concatenate()([demographic_model.layers[-2].output, transaction_model.layers[-2].output])
x = Dense(32,)(x)
x = Dense(16,)(x)
x = Dense(1)(x)
model = Model(inputs=[demographic_model.input, transaction_model.input], outputs=x)

In [57]:
model.compile(optimizer='adam', loss=SMAPE_loss(train_scaler))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv1d_22_input (InputLayer)    (None, 181, 4)       0                                            
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 179, 64)      832         conv1d_22_input[0][0]            
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 177, 64)      12352       conv1d_22[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_13 (MaxPooling1D) (None, 88, 64)       0           conv1d_23[0][0]                  
____________________________________________________________________________________________

In [58]:
log_dir = 'logs/demo_kplus_cc_conv'

model.fit([demo_x_train, transaction_x_train], y_train,
          validation_data=([demo_x_val, transaction_x_val], y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir),
                     ModelCheckpoint(log_dir + '/ep{epoch}-val_SMAPE:{val_SMAPE}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 17s 434us/step - loss: 0.0893 - val_loss: 0.0909
Epoch 1 | val-SMAPE: 90.91143378138543
Epoch 2/100
40000/40000 [==============================] - 16s 401us/step - loss: 0.0898 - val_loss: 0.0866
Epoch 2 | val-SMAPE: 91.3358620095253
Epoch 3/100
40000/40000 [==============================] - 16s 400us/step - loss: 0.0872 - val_loss: 0.0863
Epoch 3 | val-SMAPE: 91.36757788538932
Epoch 4/100
40000/40000 [==============================] - 16s 400us/step - loss: 0.0869 - val_loss: 0.0864
Epoch 4 | val-SMAPE: 91.36087724685669
Epoch 5/100
40000/40000 [==============================] - 16s 401us/step - loss: 0.0856 - val_loss: 0.0913
Epoch 5 | val-SMAPE: 90.86773957848548
Epoch 6/100
40000/40000 [==============================] - 16s 399us/step - loss: 0.0856 - val_loss: 0.0863
Epoch 6 | val-SMAPE: 91.36810836315155
Epoch 7/100
40000/40000 [==============================] - 16s 399us/

KeyboardInterrupt: ignored

In [0]:
demo_test_xs, transaction_test_xs = create_all_x_data(test_set['id'].to_numpy())

In [0]:
ys_pred = model.predict([demo_test_xs, transaction_test_xs])

In [0]:
test_incomes = train_scaler.inverse_transform(ys_pred)

In [0]:
answer = test_set.copy()
answer['income'] = test_incomes
answer.to_csv('answer.csv')